In [1]:
# -------------------------------------------------------------------------------------------------
# PPG data in the dataset are extracted (256 data points)
# -------------------------------------------------------------------------------------------------

In [73]:
import pickle

In [3]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [4]:
# Import Libraries
import platform, os, openpyxl, requests, bs4
import datetime as dt
import re
import wfdb
import pandas as pd
import numpy as np
import datetime as dt
import random
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.signal import find_peaks



In [5]:
# Define a function to read MIMIC3 data   
def read_MIMIC3_data(p_num: str, p_id: str, filename: str):
    # Read the data via internet (wfdb)
    record = wfdb.rdrecord( filename, pn_dir = 'mimic3wdb/matched/p{}/p{}/'.format(p_num, p_id))
    # Plot the data in minutes
    # wfdb.plot_wfdb(record=record, figsize=(6,12), time_units='minutes')
    # -------------------------------------- form a dataframe --------------------------------------------
    # Form the name list
    name = record.sig_name
    # Form the unit list
    unit = record.units
    # Initialize the name unit list for the dataframe
    name_unit = []
    
    newlist = []

    if len(name) == len(unit):
        for i in range(len(name)):
            name_unit.append(name[i] + ' (' + unit[i] + ')' )
            
        # Form the dataframe (columns with units
        df_nhea = pd.DataFrame( [record.p_signal.transpose().tolist()], columns = name_unit) 
        df_nhea['SUBJECT_ID'] = sub_id
        # Return the resultant dataframe
        return df_nhea
        print("The length of the name list is NOT the same as the length of the unit. ")
        return None


# Clinical part

# waveform part

In [ ]:
sub_ids = [i for i in range(30, 1000)]     # from 0 to 1_000_000

list1 = []
n = 0 

for sub_id in sub_ids:
        # Try to see whether we can access the files
    p_num = "{:02d}".format(sub_id // 10000)
    p_id = "{:06d}".format(sub_id)
    url_1 = "https://physionet.org/files/mimic3wdb/1.0/matched/p"
    url_2 = "/p"
    r = requests.get(url_1 + p_num + url_2 + p_id + "/")

        # If we can access the files
    if bool(r) == True:
            # ref: https://stackoverflow.com/questions/11023530/python-to-list-http-files-and-directories
            # Extract all the ".hea" files
        soup = bs4.BeautifulSoup(r.text, 'html.parser')
        ext = ".hea"
        filels = [node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
            # Separate the ".hea" files into 4 lists
        filels_short_hea = [x for x in filels if re.search('\d\d\d\d\d\d\d_\d\d\d\d.hea', x)]
        filels_session = np.unique([i[:-9] for i in [x for x in filels if re.search('\d\d\d\d\d\d\d_\d\d\d\d.hea', x)]])
        filels_layout = [x for x in filels if re.search('\d\d\d\d\d\d\d_layout.hea', x)]
        filels_long_hea = [x for x in filels if re.search('p\d\d\d\d\d\d-\d\d\d\d-\d\d-\d\d-\d\d-\d\d.hea', x)]
        filels_long_nhea = [x for x in filels if re.search('p\d\d\d\d\d\d-\d\d\d\d-\d\d-\d\d-\d\d-\d\dn.hea', x)]
            # ------------------------------------- Data extraction --------------------------------------------
        for item_wave in filels_short_hea:
                # Get the data
                
            session_index  = np.where(filels_session == item_wave[:-9])[0][0]
            waveform_data = wfdb.rdrecord( item_wave[:-4], pn_dir = 'mimic3wdb/matched/p{}/p{}/'.format(p_num, p_id))
            waveform_session_data = wfdb.rdrecord( filels_long_hea[session_index][:-4], pn_dir = 'mimic3wdb/matched/p{}/p{}/'.format(p_num, p_id))
            #numeric_data = wfdb.rdrecord( item_numeric[:-4], pn_dir = 'mimic3wdb/matched/p{}/p{}/'.format(p_num, p_id))
                # If PLETH is in record.sig_name
            if set(["PLETH"]).issubset( set(waveform_data.sig_name) ) or set(["PLETH"]).issubset( set(waveform_session_data.sig_name) ):
                n += 1
                waveform_df = read_MIMIC3_data(p_num = p_num, p_id = p_id, filename = item_wave[:-4])
                # numeric_df = read_MIMIC3_data(p_num = p_num, p_id = p_id, filename = item_numeric[:-4])
                waveform_df.to_pickle(r"C:\Users\User\Downloads\MIMICIII-Waveform\signs\p{}_{}_{}.pkl".format(p_id, item_wave,filels_long_hea[session_index][8:-4] ))
                # numeric_df.to_pickle(r"/Users/kinho123/Downloads/MIMICIII-Waveform/numerics/p{}_{}.pkl".format(p_id, item_numeric))
                print( f"p{p_id}_" + item_wave[:-4])
                print(n)
                print("pkl file generated at" + r"/Users/kinho123/Downloads/MIMICIII-Waveform/{}.pkl".format(item_wave) )
            else:
                print(f"p{p_id}_{item_wave}","No PPG")



p000030_3524877_0001.hea No PPG
p000033_3713820_0001.hea No PPG
p000033_3713820_0002.hea No PPG
p000033_3713820_0003.hea No PPG
p000033_3737936_0001.hea No PPG
p000052_3238451_0001
1
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3238451_0001.hea.pkl
p000052_3238451_0002
2
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3238451_0002.hea.pkl
p000052_3238451_0003
3
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3238451_0003.hea.pkl
p000052_3238451_0004
4
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3238451_0004.hea.pkl
p000052_3238451_0005
5
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3238451_0005.hea.pkl
p000052_3238451_0006
6
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3238451_0006.hea.pkl
p000052_3533390_0001
7
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3533390_0001.hea.pkl
p000052_3533390_0002
8
pkl file generated at/Users/kinho123/Downloads/MIMICII

p000107_3746356_0024
60
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3746356_0024.hea.pkl
p000107_3746356_0025
61
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3746356_0025.hea.pkl
p000107_3746356_0026
62
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3746356_0026.hea.pkl
p000107_3746356_0027
63
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3746356_0027.hea.pkl
p000107_3746356_0028
64
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3746356_0028.hea.pkl
p000107_3746356_0029
65
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3746356_0029.hea.pkl
p000107_3746356_0030
66
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3746356_0030.hea.pkl
p000107_3805787_0001
67
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0001.hea.pkl
p000107_3805787_0002
68
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0002.hea.pkl
p000107_38

p000107_3805787_0069
135
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0069.hea.pkl
p000107_3805787_0070
136
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0070.hea.pkl
p000107_3805787_0071
137
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0071.hea.pkl
p000107_3805787_0072
138
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0072.hea.pkl
p000107_3805787_0073
139
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0073.hea.pkl
p000107_3805787_0074
140
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0074.hea.pkl
p000107_3805787_0075
141
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0075.hea.pkl
p000107_3805787_0076
142
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0076.hea.pkl
p000107_3805787_0077
143
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0077.hea.pkl
p

p000107_3805787_0143
209
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0143.hea.pkl
p000107_3805787_0144
210
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0144.hea.pkl
p000107_3805787_0145
211
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0145.hea.pkl
p000107_3805787_0146
212
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0146.hea.pkl
p000107_3805787_0147
213
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0147.hea.pkl
p000107_3805787_0148
214
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0148.hea.pkl
p000107_3805787_0149
215
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0149.hea.pkl
p000107_3805787_0150
216
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0150.hea.pkl
p000107_3805787_0151
217
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3805787_0151.hea.pkl
p

p000107_3860035_0005
283
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3860035_0005.hea.pkl
p000107_3860035_0006
284
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3860035_0006.hea.pkl
p000107_3860035_0007.hea No PPG
p000107_3860035_0008
285
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3860035_0008.hea.pkl
p000107_3860035_0009.hea No PPG
p000107_3860035_0010
286
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3860035_0010.hea.pkl
p000107_3860035_0011.hea No PPG
p000107_3860035_0012
287
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3860035_0012.hea.pkl
p000107_3860035_0013.hea No PPG
p000107_3860035_0014
288
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3860035_0014.hea.pkl
p000107_3860035_0015.hea No PPG
p000107_3860035_0016
289
pkl file generated at/Users/kinho123/Downloads/MIMICIII-Waveform/3860035_0016.hea.pkl
p000107_3860035_0017.hea No PPG
p000107_3860035_0018
290
pkl fi

In [61]:
for item_wave, item_numeric in zip([i for i in range(10)], [1]):
print(item_wave, item_numeric)

IndentationError: expected an indented block (<ipython-input-61-6ee2cea22d9d>, line 2)

In [35]:
read_MIMIC3_data(p_num = p_num, p_id = p_id, filename =  item_wave[:-4])

,II (mV),AVF (mV),ABP (mmHg),PAP (mmHg)
0,-72.0,-72.0,-72.0,-72.0
1,-72.0,-72.0,-72.0,-72.0
2,-72.0,-72.0,-72.0,-72.0
3,-72.0,-72.0,-72.0,-72.0
4,-24.0,-24.0,-24.0,-24.0
...,...,...,...,...
3806,-16.0,-16.0,-17.0,-16.0
3807,-16.0,-16.0,-18.0,-16.0
3808,16.0,16.0,-18.0,16.0
3809,16.0,16.0,-18.0,16.0


In [10]:
df = pd.read_csv(r"C:\Users\User\Downloads\mimic-iii-clinical-database-1.4 (1)\mimic-iii-clinical-database-1.4\D_ITEMS.csv\D_ITEMS.csv")
A = df["LABEL"].dropna()
match = [i for i in A if "Hemoglobin" in i or 'hemoglobin' in i or 'Glucose' in i or 'glucose' in i or 'Cholesterol' in i or 'cholesterol' in i ]
del match[1]
index = [df["ITEMID"][df["LABEL"] == match[i]].tolist()[0] for i in range(len(match))]
unit = [df["UNITNAME"][df["LABEL"] == match[i]].tolist()[0] for i in range(len(match))]
print(match, index, unit)

['fingerstick glucose', 'Cholesterol (<200)', 'Fingerstick Glucose', 'Glucose (70-105)', 'Hemoglobin', 'Glucose Monitor #', 'abg: glucose', 'Blood Glucose', 'BloodGlucose', 'Cholesterol (0-199)', 'Urine Glucose', 'Cholesterol', 'Glucose', 'finger stick glucose', 'Glucose_ApacheIV', 'GlucoseScore_ApacheIV', 'Glucose (whole blood) (soft)', 'Glucose finger stick', 'Boost Glucose Control (1/4)', 'Boost Glucose Control (1/2)', 'Boost Glucose Control (3/4)', 'Boost Glucose Control (Full)', 'Cholesterol', 'Glucose (serum)', 'Hemoglobin', 'Glucose (whole blood)'] [1455, 789, 807, 811, 814, 3447, 1812, 3744, 3745, 3748, 3816, 1524, 1529, 2338, 227015, 227016, 228388, 225664, 227976, 227977, 227978, 227979, 1524, 220621, 814, 226537] [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'None', 'None', 'None', 'None', 'mL', 'mL', 'mL', 'mL', nan, 'None', nan, 'None']


In [11]:
df1 = pd.read_pickle(r"C:\Users\User\Downloads\MIMIC.pkl")
df1

,SUBJECT_ID,ITEMID,CHARTTIME,VALUENUM,VALUEUOM
0,36,223834,2134-05-12 12:00:00,15.00,L/min
1,36,223835,2134-05-12 12:00:00,100.00,NaN
2,36,224328,2134-05-12 12:00:00,0.37,NaN
3,36,224329,2134-05-12 12:00:00,6.00,min
4,36,224330,2134-05-12 12:00:00,2.50,NaN
...,...,...,...,...,...
330712478,99781,224847,2133-08-02 08:30:00,NaN,NaN
330712479,99781,224889,2133-08-02 08:30:00,NaN,NaN
330712480,99781,224903,2133-08-02 08:30:00,NaN,NaN
330712481,99781,224910,2133-08-02 08:30:00,NaN,NaN


In [12]:
biomk = [df1.index[df1['ITEMID'] == index[i]].tolist() for i in range(len(index))]

In [13]:
df2 = df1[(df1.ITEMID == index[0])|(df1.ITEMID == index[2])|(df1.ITEMID == index[3])|(df1.ITEMID == index[5])
          |(df1.ITEMID == index[6])| (df1.ITEMID == index[7]) | (df1.ITEMID == index[8]) | (df1.ITEMID == index[11]) 
          | (df1.ITEMID == index[12]) | (df1.ITEMID == index[13]) | (df1.ITEMID == index[14])| (df1.ITEMID == index[15])
          | (df1.ITEMID == index[16]) | (df1.ITEMID == index[17])| (df1.ITEMID == index[23])| (df1.ITEMID == index[25])]
df3 =  df1[(df1.ITEMID == index[1])|(df1.ITEMID == index[9])|(df1.ITEMID == index[11])|(df1.ITEMID == index[22])]
df4 =  df1[(df1.ITEMID == index[4])|(df1.ITEMID == index[24])]

In [40]:
df2[df2.SUBJECT_ID == 20]

,SUBJECT_ID,ITEMID,CHARTTIME,VALUENUM,VALUEUOM
34028881,20,811,2183-04-29 02:14:00,150.0,NaN
34042369,20,811,2183-04-29 06:00:00,122.0,NaN
34046857,20,811,2183-04-28 21:56:00,199.0,NaN
34054019,20,811,2183-04-29 00:00:00,193.0,NaN
34055871,20,811,2183-04-29 11:00:00,138.0,NaN
34060297,20,811,2183-04-29 01:00:00,181.0,NaN
34073756,20,811,2183-04-29 04:00:00,131.0,NaN
34073803,20,811,2183-04-29 05:00:00,128.0,NaN
34078207,20,811,2183-04-28 20:30:00,137.0,NaN
34078214,20,811,2183-04-28 20:41:00,193.0,NaN


### Glucose level

In [14]:
df2_clean = df2.drop_duplicates('SUBJECT_ID', keep='first')
df2_clean = df2_clean.rename(columns = {"VALUENUM": "Glucose"})
df2_clean = df2_clean.drop(columns = ["VALUEUOM"])
Sub = df2_clean["SUBJECT_ID"].unique()
df2_clean = df2_clean.reset_index(drop=True)
df2_clean = df2_clean.dropna()
df2_clean

,SUBJECT_ID,ITEMID,CHARTTIME,Glucose
0,23,225664,2157-10-22 10:00:00,188.0
1,36,220621,2134-05-12 08:33:00,126.0
2,34,225664,2191-02-23 08:00:00,133.0
3,85,220621,2167-07-27 04:38:00,112.0
4,107,225664,2121-12-01 20:00:00,110.0
...,...,...,...,...
44923,32789,811,2115-11-30 03:26:00,103.0
44924,32807,811,2173-10-28 10:12:00,126.0
44926,32779,807,2167-10-21 04:00:00,139.0
44927,32777,811,2113-10-12 22:00:00,141.0


In [15]:
data2 = {'list' : [df2["VALUENUM"][df2.SUBJECT_ID == Sub[i]].tolist() for i in range(len(Sub))]}
testdf2 = pd.DataFrame.from_dict(data2)

In [16]:
df2_final = pd.concat([df2_clean, testdf2], axis=1)
df2_final = df2_final.drop(columns=["Glucose", "ITEMID"])
df2_final = df2_final.rename(columns={"list": "Glucose Level (mg/dL)"})
df2_final = df2_final.dropna()
df2_final.index = df2_final["SUBJECT_ID"]
df2_final = df2_final.drop(columns=["SUBJECT_ID"])
df2_final

,CHARTTIME,Glucose Level (mg/dL)
SUBJECT_ID,,
23.0,2157-10-22 10:00:00,"[188.0, 136.0, 123.0, 105.0, 114.0, 110.0, 111..."
36.0,2134-05-12 08:33:00,"[126.0, 92.0, 101.0, 105.0, 121.0, 111.0, 100...."
34.0,2191-02-23 08:00:00,"[133.0, 116.0, 103.0, 118.0, 164.0, 164.0, 109..."
85.0,2167-07-27 04:38:00,"[112.0, 114.0, 117.0, 111.0, 111.0, 130.0, 130..."
107.0,2121-12-01 20:00:00,"[110.0, 90.0, 79.0, 125.0, 90.0, 162.0, 88.0, ..."
...,...,...
32789.0,2115-11-30 03:26:00,"[103.0, 103.0, 105.0, 105.0, 109.0, 123.0, 112..."
32807.0,2173-10-28 10:12:00,"[126.0, 126.0, 78.0, 78.0, 150.0, 150.0, 130.0..."
32779.0,2167-10-21 04:00:00,"[139.0, 150.0, 174.0, 177.0, 165.0, 70.0, 71.0..."


### Cholesterol

In [17]:
df3_clean = df3.drop_duplicates('SUBJECT_ID', keep='first')
df3_clean = df3_clean.rename(columns = {"VALUENUM": "Cholesterol"})
Sub1 = df3_clean["SUBJECT_ID"].unique()
df3_clean = df3_clean.reset_index(drop=True)

In [18]:
data3 = {'list' : [df3["VALUENUM"][df3.SUBJECT_ID == Sub1[i]].tolist() for i in range(len(Sub1))]}
testdf3 = pd.DataFrame.from_dict(data3)

In [19]:
df3_final = pd.concat([df3_clean, testdf3], axis=1)
df3_final = df3_final.drop(columns=["VALUEUOM", "Cholesterol", "ITEMID"])
df3_final = df3_final.rename(columns={"list": "Cholesterol Level (mg/dL)", "ITEMID": "ITEMID2"})
df3_final = df3_final.dropna()
df3_final.index = df3_final["SUBJECT_ID"]
df3_final = df3_final.drop(columns=["SUBJECT_ID"])
df3_final

,CHARTTIME,Cholesterol Level (mg/dL)
SUBJECT_ID,,
9,2149-11-10 03:00:00,"[208.0, 208.0]"
13,2167-01-09 07:11:00,"[252.0, 252.0]"
44,2192-11-20 06:10:00,"[139.0, 139.0]"
42,2116-04-26 20:00:00,"[178.0, 178.0]"
79,2175-09-26 01:45:00,"[164.0, 164.0]"
...,...,...
32684,2157-07-01 03:05:00,"[103.0, 103.0]"
32788,2128-01-01 05:00:00,"[91.0, 91.0]"
32805,2182-11-15 03:40:00,"[148.0, 148.0]"


### Hemoglobin

In [20]:
df4_clean = df4.drop_duplicates('SUBJECT_ID', keep='first')
df4_clean = df4_clean.rename(columns = {"VALUENUM": "Hemoglobin"})
Sub2 = df4_clean["SUBJECT_ID"].unique()
df4_clean = df4_clean.reset_index(drop=True)

In [21]:
data4 = {'list' : [df4["VALUENUM"][df4.SUBJECT_ID == Sub1[i]].tolist() for i in range(len(Sub1))]}
testdf4 = pd.DataFrame.from_dict(data4)

In [22]:
df4_final = pd.concat([df4_clean, testdf4], axis=1)
df4_final = df4_final.drop(columns=["VALUEUOM","Hemoglobin", "ITEMID"])
df4_final = df4_final.rename(columns={"list": "Hemoglobin Level (gm/dL)"})
df4_final = df4_final.dropna()
df4_final.index = df4_final["SUBJECT_ID"]
df4_final = df4_final.drop(columns=["SUBJECT_ID"])
df4_final

,CHARTTIME,Hemoglobin Level (gm/dL)
SUBJECT_ID,,
26,2126-05-09 04:30:00,"[13.100000381469728, 12.600000381469728, 12.80..."
20,2183-04-29 02:14:00,"[12.300000190734863, 10.199999809265137, 10.80..."
3,2101-10-22 01:00:00,"[8.899999618530273, 10.5, 9.899999618530272, 8..."
25,2160-11-02 17:00:00,"[12.5, 11.699999809265137, 12.100000381469728]"
31,2108-08-24 19:30:00,"[13.399999618530272, 13.399999618530272]"
...,...,...
3327,2107-08-18 01:40:00,"[8.0, 9.899999618530272, 9.100000381469727, 9...."
3302,2171-11-07 04:30:00,"[9.399999618530272, 10.0, 9.199999809265137, 9..."
3314,2150-11-19 09:57:00,"[11.199999809265137, 11.199999809265137, 11.60..."


In [23]:
df_new = pd.concat([df2_final, df3_final, df4_final], axis=1)
df_new

,CHARTTIME,Glucose Level (mg/dL),CHARTTIME,Cholesterol Level (mg/dL),CHARTTIME,Hemoglobin Level (gm/dL)
SUBJECT_ID,,,,,,
3.0,2101-10-22 04:00:00,"[126.0, 133.0, 99.0, 267.0, 120.0, 128.0, 306....",NaN,NaN,2101-10-22 01:00:00,"[8.899999618530273, 10.5, 9.899999618530272, 8..."
4.0,2191-03-16 08:00:00,"[266.0, 205.0, 205.0, 153.0, 183.0, 183.0, 225...",NaN,NaN,2191-03-16 05:42:00,"[12.699999809265137, 6.400000095367432, 6.5999..."
6.0,2175-06-02 10:00:00,"[89.0, 98.0, 81.0, 118.0, 118.0, 181.0, 181.0,...",NaN,NaN,2175-06-03 00:58:00,[]
9.0,2149-11-11 18:00:00,"[178.0, 176.0, 170.0, 184.0, 116.0, 114.0, 170...",2149-11-10 03:00:00,"[208.0, 208.0]",2149-11-12 03:00:00,"[13.300000190734863, 13.199999809265137]"
11.0,2178-04-17 02:35:00,"[126.0, 126.0, 120.0, 119.0, 130.0, 130.0, 149...",NaN,NaN,2178-04-17 02:35:00,"[14.5, 14.199999809265137]"
...,...,...,...,...,...,...
99985.0,2181-02-03 04:29:00,"[118.0, 144.0, 144.0, 151.0, 96.0, 168.0, 117....",NaN,NaN,NaN,NaN
99991.0,2184-12-31 08:00:00,"[132.0, 132.0, 155.0, 121.0, 137.0, 127.0, 145...",NaN,NaN,NaN,NaN
99992.0,2144-07-26 04:38:00,"[110.0, 123.0, 126.0, 161.0, 168.0]",NaN,NaN,NaN,NaN


In [24]:
pd.read_csv(r"C:\Users\User\Downloads\mimic-iii-clinical-database-1.4 (1)\mimic-iii-clinical-database-1.4\PATIENTS.csv\PATIENTS.csv")

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
46515,31840,44089,M,2026-05-25 00:00:00,NaN,NaN,NaN,0
46516,31841,44115,F,2124-07-27 00:00:00,NaN,NaN,NaN,0
46517,31842,44123,F,2049-11-26 00:00:00,2135-01-12 00:00:00,2135-01-12 00:00:00,NaN,1
46518,31843,44126,F,2076-07-25 00:00:00,NaN,NaN,NaN,0


In [25]:
df_10 = pd.read_csv(r"C:\Users\User\Downloads\mimic-iii-clinical-database-1.4 (1)\mimic-iii-clinical-database-1.4\DATETIMEEVENTS.csv\DATETIMEEVENTS.csv", usecols = ["CHARTTIME", "STORETIME", "SUBJECT_ID"])